In [1]:
import sys
import os

# Defina o caminho base manualmente
base_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))

# Adicione o diretório base ao caminho de pesquisa de módulos
sys.path.append(base_path)

# Agora você pode importar utils e outros módulos
#import utils
#import setups.emas as emas
#import setups.stopgain as StopGain
#import setups.stoploss as StopLoss

from evaluated_strategy import EvaluatedStrategy
from evaluator import StrategyEvaluator

import numpy as np
import itertools
import pandas as pd
from datetime import datetime
from datetime import datetime, timedelta

In [2]:
def adjust_date(start_date):
    start_datetime = datetime.strptime(start_date, '%Y-%m-%d')
    days_to_subtract = 10.40625
    new_datetime = start_datetime - timedelta(days=days_to_subtract)
    return new_datetime.strftime('%Y-%m-%d')

def calculate_sharpe_ratio(returns, risk_free_rate=0.05):
    excess_returns = returns - risk_free_rate
    mean_excess_return = np.mean(excess_returns)
    std_excess_return = np.std(excess_returns)
    sharpe_ratio = mean_excess_return / std_excess_return if std_excess_return != 0 else 0
    return sharpe_ratio

In [16]:
# Carregar os dados dos arquivos CSV
df_15m = pd.read_csv('BTC_15m_candles.csv')
df_5m = pd.read_csv('BTC_5m_candles.csv')

# Padronizar os nomes das colunas para minúsculas e substituir espaços por underscores
df_15m.columns = [col.lower().replace(' ', '_') for col in df_15m.columns]
df_5m.columns = [col.lower().replace(' ', '_') for col in df_5m.columns]

# Definir o tempo inicial e o tempo final
start_date = '2023-08-11'
end_date = '2024-08-01'

adjusted_start_date = adjust_date(start_date)

# Converter strings de data para datetime
start_datetime = pd.to_datetime(start_date)
end_datetime = pd.to_datetime(end_date)

# Filtrar os dados com base no tempo inicial e final
df_15m['open_time'] = pd.to_datetime(df_15m['open_time'])
df_5m['open_time'] = pd.to_datetime(df_5m['open_time'])
df_15m = df_15m[(df_15m['open_time'] >= start_datetime) & (df_15m['open_time'] <= end_datetime)]
df_5m = df_5m[(df_5m['open_time'] >= start_datetime) & (df_5m['open_time'] <= end_datetime)]

In [17]:
# Definir os parâmetros e seus intervalos
param_grid = {
#    'short_period': list(range(5, 10, 1)),
#    'long_period': list(range(44, 61, 1)),
#    'ratio': np.round(np.arange(1.0, 4.0, 0.1), 1).tolist(),
    'timeframe': ['15m'],
#    'stop_candles': list(range(20, 30, 1)),
    'ativo': ['BTCUSDT'],
    'setup': ['EMA'],
    'short_period': list(range(4, 6, 1)),
    'long_period': list(range(14, 16, 1)),
    'ratio': [4.1],
    'stop_candles': list(range(16, 18, 1)),
}

param_combinations = list(itertools.product(*param_grid.values()))

In [18]:
strategies = []
starting_balance = 1000
trading_tax = 0.03

if param_grid['timeframe'] == ['15m']:
    data = df_15m.copy()
else:
    data = df_5m.copy()

for combination in param_combinations:
    params = dict(zip(param_grid.keys(), combination))
    strategy = EvaluatedStrategy(params, starting_balance, trading_tax, data, params['short_period'], params['long_period'], params['stop_candles'], params['ratio'])
    strategies.append(strategy)

evaluator = StrategyEvaluator(data, strategies, lambda strategy: strategy.balance - 1000, 999)
while not evaluator.is_done_evaluating():
    evaluator.evaluate_next_candle()
results = evaluator.results()
best_performance = results['performance']
best_params = results['identifier']
best_result = {
    'final_saldo': results['metrics']['balance'],
    'trades': results['metrics']['trades'],
    'results': results['metrics']['monthly_results'],
    'max_drawdown': results['metrics']['max_drawdown'],
    'ganhos': results['metrics']['gains'],
    'perdas': results['metrics']['losses']
}
# Imprimir os resultados detalhados do melhor desempenho
if best_result:
    saldo = best_result['final_saldo']
    trades = best_result['trades']
    results = best_result['results']
    max_drawdown = best_result['max_drawdown']
    ganhos = best_result['ganhos']
    perdas = best_result['perdas']

    overall_sharpe_ratio = calculate_sharpe_ratio(np.array(ganhos + perdas), 0.15)

    for year in results:
        print(f"Ano: {year}")
        print(f"  Operações realizadas: {sum([results[year][month]['open_trades'] for month in results[year]])}")
        print(f"  Trades de sucesso: {sum([results[year][month]['successful_trades'] for month in results[year]])}")
        print(f"  Soma dos ganhos: {sum([results[year][month]['lucro'] for month in results[year]]):.2f}%")
        try:
            print(f"  Ganho médio por trade: {sum([results[year][month]['lucro'] for month in results[year]]) / sum([results[year][month]['successful_trades'] for month in results[year]]) :.2f}%")
        except ZeroDivisionError:
            print(f"  Ganho médio por trade: 0")
        print(f"  Trades em prejuízo: {sum([results[year][month]['failed_trades'] for month in results[year]])}")
        print(f"  Soma das perdas: {sum([results[year][month]['perda_percentual_total'] for month in results[year]]):.2f}%")
        
        total_loss = sum([results[year][month]['perda_percentual_total'] for month in results[year]])
        total_failed_trades = sum([results[year][month]['failed_trades'] for month in results[year]])

        if total_failed_trades != 0:
            avg_loss_per_trade = total_loss / total_failed_trades
        else:
            avg_loss_per_trade = 0

        print(f"  Perda média por trade: {avg_loss_per_trade:.2f}%")
        
        if results[year][list(results[year].keys())[0]]['saldo_inicial'] <= results[year][list(results[year].keys())[-1]]['saldo_final']:
            print(f"  Resultado final: {((results[year][list(results[year].keys())[-1]]['saldo_final'] / results[year][list(results[year].keys())[0]]['saldo_inicial']) - 1) * 100:.2f}%")
        else:
            print(f"  Resultado final: {((1 - (results[year][list(results[year].keys())[-1]]['saldo_final'] / results[year][list(results[year].keys())[0]]['saldo_inicial'])) * -1) * 100:.2f}%")
        
        print(f"  Saldo inicial: {results[year][list(results[year].keys())[0]]['saldo_inicial']:.2f}")
        print(f"  Saldo final: {results[year][list(results[year].keys())[-1]]['saldo_final']:.2f}")
        print("Detalhes:")
        for month in results[year]:
            print(f"  Mês: {month}")
            print(f"    Operações realizadas: {results[year][month]['open_trades']}")
            print(f"    Trades de sucesso: {results[year][month]['successful_trades']}")
            print(f"    Soma dos ganhos: {results[year][month]['lucro']:.2f}%")
            try:
                print(f"    Ganho médio por trade: {results[year][month]['lucro'] / results[year][month]['successful_trades']:.2f}%")
            except ZeroDivisionError:
                print(f"    Ganho médio por trade: 0")
            print(f"    Trades em prejuízo: {results[year][month]['failed_trades']}")
            print(f"    Soma das perdas: {results[year][month]['perda_percentual_total']:.2f}%")
            
            failed_trades = results[year][month]['failed_trades']

            if failed_trades != 0:
                avg_loss_per_trade = results[year][month]['perda_percentual_total'] / failed_trades
            else:
                avg_loss_per_trade = 0

            print(f"    Perda média por trade: {avg_loss_per_trade:.2f}%")
            print(f"    Drawdown máximo: {results[year][month]['max_drawdown']:.2f}%")

            if results[year][month]['saldo_inicial'] <= results[year][month]['saldo_final']:
                print(f"    Resultado final: {(results[year][month]['saldo_final'] / results[year][month]['saldo_inicial'] - 1) * 100:.2f}%")
            else:
                print(f"    Resultado final: {((1 - (results[year][month]['saldo_final'] / results[year][month]['saldo_inicial'])) * -1) * 100:.2f}%")

            print(f"    Saldo inicial: {results[year][month]['saldo_inicial']:.2f}")
            print(f"    Saldo final: {results[year][month]['saldo_final']:.2f}")
            print("-------------------")

    print("Total:")
    print(f"Operações realizadas: {sum([results[year][month]['open_trades'] for year in results for month in results[year]])}")
    print(f"Sharpe Ratio: {overall_sharpe_ratio:.2f}")
    try:
        print(f"Taxa de acerto: {sum([results[year][month]['successful_trades'] for year in results for month in results[year]]) / sum([results[year][month]['open_trades'] for year in results for month in results[year]]) * 100:.2f}%")
    except ZeroDivisionError:
        print(f"Taxa de acerto: 0")
    print(f"Trades de sucesso: {sum([results[year][month]['successful_trades'] for year in results for month in results[year]])}")
    print(f"Soma dos ganhos: {sum([results[year][month]['lucro'] for year in results for month in results[year]]):.2f}%")

    try:
        print(f"Ganho médio por trade: {sum([results[year][month]['lucro'] for year in results for month in results[year]]) / sum([results[year][month]['successful_trades'] for year in results for month in results[year]]) :.2f}%")
    except ZeroDivisionError:
        print(f"Ganho médio por trade: 0")

    print(f"Trades em prejuízo: {sum([results[year][month]['failed_trades'] for year in results for month in results[year]])}")
    print(f"Soma das perdas: {sum([results[year][month]['perda_percentual_total'] for year in results for month in results[year]]):.2f}%")
    try:
        print(f"Perda média por trade: {sum([results[year][month]['perda_percentual_total'] for year in results for month in results[year]]) / sum([results[year][month]['failed_trades'] for year in results for month in results[year]]) :.2f}%")
    except ZeroDivisionError:
        print(f"Perda média por trade: 0")
    print(f"Drawdown máximo: {max_drawdown:.2f}%")

    saldo_inicial = results[list(results.keys())[0]][list(results[list(results.keys())[0]].keys())[0]]['saldo_inicial']
    saldo_final = results[list(results.keys())[-1]][list(results[list(results.keys())[-1]].keys())[-1]]['saldo_final']

    if saldo_inicial <= saldo_final:
        print(f"Resultado final: {(saldo_final / saldo_inicial - 1) * 100:.2f}%")
    else:
        print(f"Resultado final: {((1 - (saldo_final / saldo_inicial)) * -1) * 100:.2f}%")

    print(f"Saldo inicial: {results[list(results.keys())[0]][list(results[list(results.keys())[0]].keys())[0]]['saldo_inicial']:.2f}")
    print(f"Saldo final: {results[list(results.keys())[-1]][list(results[list(results.keys())[-1]].keys())[-1]]['saldo_final']:.2f}")
    print("-------------------")
    print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}: Teste finalizado: {params['ativo']} - {params['timeframe']}.")
    print(f"Setup: EMA {params['short_period']}/{params['long_period']} rompimento, stopgain ratio {params['ratio']} e stoploss {params['stop_candles']} candles")

Ano: 2023
  Operações realizadas: 25
  Trades de sucesso: 10
  Soma dos ganhos: 72.21%
  Ganho médio por trade: 7.22%
  Trades em prejuízo: 14
  Soma das perdas: 19.95%
  Perda média por trade: 1.42%
  Resultado final: 60.52%
  Saldo inicial: 1000.00
  Saldo final: 1605.24
Detalhes:
  Mês: 8
    Operações realizadas: 6
    Trades de sucesso: 1
    Soma dos ganhos: 8.56%
    Ganho médio por trade: 8.56%
    Trades em prejuízo: 5
    Soma das perdas: 9.94%
    Perda média por trade: 1.99%
    Drawdown máximo: 6.95%
    Resultado final: -2.15%
    Saldo inicial: 1000.00
    Saldo final: 978.51
-------------------
  Mês: 9
    Operações realizadas: 15
    Trades de sucesso: 5
    Soma dos ganhos: 14.57%
    Ganho médio por trade: 2.91%
    Trades em prejuízo: 9
    Soma das perdas: 10.00%
    Perda média por trade: 1.11%
    Drawdown máximo: 10.35%
    Resultado final: 3.90%
    Saldo inicial: 978.51
    Saldo final: 1016.62
-------------------
  Mês: 10
    Operações realizadas: 3
    Tra

In [6]:
print("Melhores parâmetros encontrados:")
print(best_params)
print("Melhor desempenho obtido:")
print(best_performance)

Melhores parâmetros encontrados:
{'timeframe': '15m', 'ativo': 'BTCUSDT', 'setup': 'EMA', 'short_period': 5, 'long_period': 15, 'ratio': 4.0, 'stop_candles': 17}
Melhor desempenho obtido:
1431.5927816954404


In [7]:
''' melhor resultado gridsearch até o momento
Melhores parâmetros encontrados:
{'short_period': 5, 'long_period': 15, 'ratio': 4.1, 'timeframe': '15m', 'stop_candles': 17, 'ativo': 'BTCUSDT', 'setup': 'EMA'}
Melhor desempenho obtido:
2005.7657471035072

periodo: 2023-08-01 ~ 2024-08-01
-------------------
Total:
Operações realizadas: 79
Sharpe Ratio: 0.27
Taxa de acerto: 32.91%
Trades de sucesso: 26
Soma dos ganhos: 196.93%
Ganho médio por trade: 7.57%
Trades em prejuízo: 52
Soma das perdas: 74.96%
Perda média por trade: 1.44%
Drawdown máximo: 16.72%
Resultado final: 200.58%
Saldo inicial: 1000.00
Saldo final: 3005.77
-------------------
2024-08-08 04:20:20: Teste finalizado: BTCUSDT - 15m.

periodo: 2020-08-01 ~ 2024-08-01
-------------------
Total:
Operações realizadas: 313
Sharpe Ratio: 0.11
Taxa de acerto: 24.92%
Trades de sucesso: 78
Soma dos ganhos: 855.50%
Ganho médio por trade: 10.97%
Trades em prejuízo: 234
Soma das perdas: 555.53%
Perda média por trade: 2.37%
Drawdown máximo: 66.27%
Resultado final: 776.62%
Saldo inicial: 1000.00
Saldo final: 8766.23
-------------------
2024-08-08 16:49:06: Teste finalizado: BTCUSDT - 15m.
Setup: ema 5/15 rompimento, stopgain ratio 4.1 e stoploss 17 candles
'''

" melhor resultado gridsearch até o momento\nMelhores parâmetros encontrados:\n{'short_period': 5, 'long_period': 15, 'ratio': 4.1, 'timeframe': '15m', 'stop_candles': 17, 'ativo': 'BTCUSDT', 'setup': 'EMA'}\nMelhor desempenho obtido:\n2005.7657471035072\n\nperiodo: 2023-08-01 ~ 2024-08-01\n-------------------\nTotal:\nOperações realizadas: 79\nSharpe Ratio: 0.27\nTaxa de acerto: 32.91%\nTrades de sucesso: 26\nSoma dos ganhos: 196.93%\nGanho médio por trade: 7.57%\nTrades em prejuízo: 52\nSoma das perdas: 74.96%\nPerda média por trade: 1.44%\nDrawdown máximo: 16.72%\nResultado final: 200.58%\nSaldo inicial: 1000.00\nSaldo final: 3005.77\n-------------------\n2024-08-08 04:20:20: Teste finalizado: BTCUSDT - 15m.\n\nperiodo: 2020-08-01 ~ 2024-08-01\n-------------------\nTotal:\nOperações realizadas: 313\nSharpe Ratio: 0.11\nTaxa de acerto: 24.92%\nTrades de sucesso: 78\nSoma dos ganhos: 855.50%\nGanho médio por trade: 10.97%\nTrades em prejuízo: 234\nSoma das perdas: 555.53%\nPerda média

In [8]:
''' melhor resultado 5m até o momento
Melhores parâmetros encontrados:
{'short_period': 7, 'long_period': 49, 'ratio': 3.0, 'timeframe': '5m', 'stop_candles': 23, 'ativo': 'BTCUSDT', 'setup': 'EMA'}
Melhor desempenho obtido:
1718.2028095527517

periodo: 2023-08-01 ~ 2024-08-01
-------------------
Total:
Operações realizadas: 206
Sharpe Ratio: 0.14
Taxa de acerto: 31.07%
Trades de sucesso: 64
Soma dos ganhos: 257.48%
Ganho médio por trade: 4.02%
Trades em prejuízo: 142
Soma das perdas: 145.62%
Perda média por trade: 1.03%
Drawdown máximo: 16.37%
Resultado final: 171.82%
Saldo inicial: 1000.00
Saldo final: 2718.20
-------------------
2024-08-28 00:04:14: Teste finalizado: BTCUSDT - 5m.
Setup: EMA 21/77 rompimento, stopgain ratio 4.8 e stoploss 23 candles

periodo: 2020-08-01 ~ 2024-08-01
-------------------
Total:
Operações realizadas: 206
Sharpe Ratio: 0.14
Taxa de acerto: 31.07%
Trades de sucesso: 64
Soma dos ganhos: 257.48%
Ganho médio por trade: 4.02%
Trades em prejuízo: 142
Soma das perdas: 145.62%
Perda média por trade: 1.03%
Drawdown máximo: 16.37%
Resultado final: 171.82%
Saldo inicial: 1000.00
Saldo final: 2718.20
-------------------
2024-08-28 00:04:14: Teste finalizado: BTCUSDT - 5m.
Setup: EMA 21/77 rompimento, stopgain ratio 4.8 e stoploss 23 candles
'''

" melhor resultado 5m até o momento\nMelhores parâmetros encontrados:\n{'short_period': 7, 'long_period': 49, 'ratio': 3.0, 'timeframe': '5m', 'stop_candles': 23, 'ativo': 'BTCUSDT', 'setup': 'EMA'}\nMelhor desempenho obtido:\n1718.2028095527517\n\nperiodo: 2023-08-01 ~ 2024-08-01\n-------------------\nTotal:\nOperações realizadas: 206\nSharpe Ratio: 0.14\nTaxa de acerto: 31.07%\nTrades de sucesso: 64\nSoma dos ganhos: 257.48%\nGanho médio por trade: 4.02%\nTrades em prejuízo: 142\nSoma das perdas: 145.62%\nPerda média por trade: 1.03%\nDrawdown máximo: 16.37%\nResultado final: 171.82%\nSaldo inicial: 1000.00\nSaldo final: 2718.20\n-------------------\n2024-08-28 00:04:14: Teste finalizado: BTCUSDT - 5m.\nSetup: EMA 21/77 rompimento, stopgain ratio 4.8 e stoploss 23 candles\n\nperiodo: 2020-08-01 ~ 2024-08-01\n-------------------\nTotal:\nOperações realizadas: 206\nSharpe Ratio: 0.14\nTaxa de acerto: 31.07%\nTrades de sucesso: 64\nSoma dos ganhos: 257.48%\nGanho médio por trade: 4.02%\

In [9]:
''' Bot LIVE atual
periodo: 2023-08-01 ~ 2024-08-01
-------------------
Total:
Operações realizadas: 91
Sharpe Ratio: 0.20
Taxa de acerto: 29.67%
Trades de sucesso: 27
Soma dos ganhos: 202.77%
Ganho médio por trade: 7.51%
Trades em prejuízo: 64
Soma das perdas: 98.33%
Perda média por trade: 1.54%
Drawdown máximo: 18.19%
Resultado final: 151.66%
Saldo inicial: 1000.00
Saldo final: 2516.65
-------------------
2024-08-08 17:14:12: Teste finalizado: BTCUSDT - 15m.
Setup: EMA 9/21 rompimento, stopgain ratio 3.5 e stoploss 14 candles

periodo: 2020-08-01 ~ 2024-08-01
-------------------
Total:
Operações realizadas: 501
Sharpe Ratio: 0.06
Taxa de acerto: 24.75%
Trades de sucesso: 124
Soma dos ganhos: 1000.71%
Ganho médio por trade: 8.07%
Trades em prejuízo: 377
Soma das perdas: 762.73%
Perda média por trade: 2.02%
Drawdown máximo: 68.08%
Resultado final: 383.73%
Saldo inicial: 1000.00
Saldo final: 4837.26
-------------------
2024-08-08 17:01:25: Teste finalizado: BTCUSDT - 15m.
Setup: EMA 9/21 rompimento, stopgain ratio 3.5 e stoploss 14 candles
'''

' Bot LIVE atual\nperiodo: 2023-08-01 ~ 2024-08-01\n-------------------\nTotal:\nOperações realizadas: 91\nSharpe Ratio: 0.20\nTaxa de acerto: 29.67%\nTrades de sucesso: 27\nSoma dos ganhos: 202.77%\nGanho médio por trade: 7.51%\nTrades em prejuízo: 64\nSoma das perdas: 98.33%\nPerda média por trade: 1.54%\nDrawdown máximo: 18.19%\nResultado final: 151.66%\nSaldo inicial: 1000.00\nSaldo final: 2516.65\n-------------------\n2024-08-08 17:14:12: Teste finalizado: BTCUSDT - 15m.\nSetup: EMA 9/21 rompimento, stopgain ratio 3.5 e stoploss 14 candles\n\nperiodo: 2020-08-01 ~ 2024-08-01\n-------------------\nTotal:\nOperações realizadas: 501\nSharpe Ratio: 0.06\nTaxa de acerto: 24.75%\nTrades de sucesso: 124\nSoma dos ganhos: 1000.71%\nGanho médio por trade: 8.07%\nTrades em prejuízo: 377\nSoma das perdas: 762.73%\nPerda média por trade: 2.02%\nDrawdown máximo: 68.08%\nResultado final: 383.73%\nSaldo inicial: 1000.00\nSaldo final: 4837.26\n-------------------\n2024-08-08 17:01:25: Teste final